In [ ]:
import yaml
import re
from textgenrnn import textgenrnn


with open("config.yml", "r") as f:
    cfg = yaml.load(f)


texts = []
context_labels = []

for user in cfg['twitter_users']:
    print("Downloading {}'s Tweets...".format(user))
    all_tweets = tweepy.Cursor(api.user_timeline,
                               screen_name=user,
                               count=200,
                               tweet_mode='extended',
                               include_rts=False).pages(16)
    for page in all_tweets:
        for tweet in page:
            tweet_text = clean_tweet(tweet.full_text)
            if tweet_text is not '':
                texts.append(tweet_text)
                context_labels.append(user)

textgen = textgenrnn(name='{}_twitter'.format("_".join(cfg['twitter_users'])))

if cfg['new_model']:
    textgen.train_new_model(
        texts,
        context_labels=context_labels,
        num_epochs=cfg['num_epochs'],
        gen_epochs=cfg['gen_epochs'],
        batch_size=cfg['batch_size'],
        train_size=cfg['train_size'],
        rnn_layers=cfg['model_config']['rnn_layers'],
        rnn_size=cfg['model_config']['rnn_size'],
        rnn_bidirectional=cfg['model_config']['rnn_bidirectional'],
        max_length=cfg['model_config']['max_length'],
        dim_embeddings=cfg['model_config']['dim_embeddings'],
        word_level=cfg['model_config']['word_level'])
else:
    textgen.train_on_texts(
        texts,
        context_labels=context_labels,
        num_epochs=cfg['num_epochs'],
        gen_epochs=cfg['gen_epochs'],
        train_size=cfg['train_size'],
        batch_size=cfg['batch_size'])

In [1]:
from textgenrnn import textgenrnn

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [10]:
textgen=textgenrnn()
textgen.train_on_texts(list, num_epochs=1)
textgen.generate(3, temperature=0.5)

Training on 4,025 character sequences.
Epoch 1/1
31/31 [==============================] - ETA: 2:12 - loss: 1.809 - ETA: 1:07 - loss: 1.939 - ETA: 46s - loss: 1.986 - ETA: 35s - loss: 1.92 - ETA: 28s - loss: 1.85 - ETA: 24s - loss: 1.77 - ETA: 20s - loss: 1.71 - ETA: 18s - loss: 1.75 - ETA: 16s - loss: 1.73 - ETA: 14s - loss: 1.78 - ETA: 13s - loss: 1.75 - ETA: 11s - loss: 1.73 - ETA: 10s - loss: 1.71 - ETA: 9s - loss: 1.6937 - ETA: 8s - loss: 1.674 - ETA: 8s - loss: 1.661 - ETA: 7s - loss: 1.640 - ETA: 6s - loss: 1.620 - ETA: 5s - loss: 1.619 - ETA: 5s - loss: 1.624 - ETA: 4s - loss: 1.611 - ETA: 4s - loss: 1.597 - ETA: 3s - loss: 1.591 - ETA: 3s - loss: 1.576 - ETA: 2s - loss: 1.559 - ETA: 2s - loss: 1.555 - ETA: 1s - loss: 1.545 - ETA: 1s - loss: 1.539 - ETA: 0s - loss: 1.522 - ETA: 0s - loss: 1.522 - 13s 412ms/step - loss: 1.5128
####################
Temperature: 0.2
####################
RT : why the fuck did tommy pickles  Pickles who the fuck did tommy pickles of pickles of the f

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Hi this girls who did the flathead eid scotioning so that a whole mechanic



 33%|████████████████████████████                                                        | 1/3 [00:01<00:02,  1.15s/it]

I therk of that pickles of pickles of this have a whole mechanic



 67%|████████████████████████████████████████████████████████                            | 2/3 [00:02<00:01,  1.11s/it]

RT : on some pickles pickles of that a whole mechanic



100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.03s/it]


In [ ]:
textgen.train_new_model(
        texts,
        context_labels=context_labels,
        num_epochs=cfg['num_epochs'],
        gen_epochs=cfg['gen_epochs'],
        batch_size=cfg['batch_size'],
        train_size=cfg['train_size'],
        rnn_layers=cfg['model_config']['rnn_layers'],
        rnn_size=cfg['model_config']['rnn_size'],
        rnn_bidirectional=cfg['model_config']['rnn_bidirectional'],
        max_length=cfg['model_config']['max_length'],
        dim_embeddings=cfg['model_config']['dim_embeddings'],
        word_level=cfg['model_config']['word_level'])

In [26]:
import json
import collections
import pandas as pd
import datetime as dt
from twitterscraper.query import query_tweets
from twitterscraper.query import query_user_info
from twitterscraper.ts_logger import logger


class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, '__json__'):
            return obj.__json__()
        elif isinstance(obj, collections.Iterable):
            return list(obj)
        elif isinstance(obj, dt.datetime):
            return obj.isoformat()
        elif hasattr(obj, '__getitem__') and hasattr(obj, 'keys'):
            return dict(obj)
        elif hasattr(obj, '__dict__'):
            return {member: getattr(obj, member)
                    for member in dir(obj)
                    if not member.startswith('_') and
                    not hasattr(getattr(obj, member), '__call__')}

        return json.JSONEncoder.default(self, obj)

def valid_date(s):
    try:
        return dt.datetime.strptime(s, "%Y-%m-%d").date()
    except ValueError:
        msg = "Not a valid date: '{0}'.".format(s)

def main(query, limit, begindate, enddate):
#         parser.add_argument("--profiles", action='store_true',
#                             help="Set this flag to if you want to scrape profile info of all the users where you" 
#                             "have previously scraped from. After all of the tweets have been scraped it will start"
#                             "a new process of scraping profile pages.")
#         parser.add_argument("--lang", type=str, default=None,
#                             help="Set this flag if you want to query tweets in \na specific language. You can choose from:\n"
#                                  "en (English)\nar (Arabic)\nbn (Bengali)\n"
#                                  "cs (Czech)\nda (Danish)\nde (German)\nel (Greek)\nes (Spanish)\n"
#                                  "fa (Persian)\nfi (Finnish)\nfil (Filipino)\nfr (French)\n"
#                                  "he (Hebrew)\nhi (Hindi)\nhu (Hungarian)\n"
#                                  "id (Indonesian)\nit (Italian)\nja (Japanese)\n"
#                                  "ko (Korean)\nmsa (Malay)\nnl (Dutch)\n"
#                                  "no (Norwegian)\npl (Polish)\npt (Portuguese)\n"
#                                  "ro (Romanian)\nru (Russian)\nsv (Swedish)\n"
#                                  "th (Thai)\ntr (Turkish)\nuk (Ukranian)\n"
#                                  "ur (Urdu)\nvi (Vietnamese)\n"
#                                  "zh-cn (Chinese Simplified)\n"
#                                  "zh-tw (Chinese Traditional)"
#                                  )
#             if args.profiles and tweets:
#                 list_users = list(set([tweet.user for tweet in tweets]))
#                 list_users_info = [query_user_info(elem) for elem in list_users]
#                 filename = 'userprofiles_' + args.output
#                 with open(filename, "w", encoding="utf-8") as output:
#                     json.dump(list_users_info, output, cls=JSONEncoder)
    tweets = twitterscraper.query.query_tweets(query, begindate, enddate, limit, lang='en')
    dictionary  = (json.dumps(tweets, cls=JSONEncoder))
    df = pd.read_json(dictionary)
    smaller_df = df[["text", "timestamp", "tweet_id", "user_id", "username"]]
    tweet_list = []
    for row in smaller_df.itertuples():
        #new_dict= {}
        #new_dict['tweet_id'] = row.tweet_id
        #new_dict['text'] = row.text
        #new_dict['time'] = row.timestamp
        #prob_dist = loaded_model.prob_classify(row.text)
        #new_dict['score'] = prob_dist.prob('pos')
        #username = row.username
        #twitterscraper.query.quer_user_info
        tweet_list.append(row.text)
    return(tweet_list)

In [28]:
import datetime as dt

begin=dt.date(2019, 7, 17)
end=dt.date(2019, 7, 23)
tweets = main("pickle", limit=20)

TypeError: main() missing 2 required positional arguments: 'begindate' and 'enddate'

In [7]:
dt.date(2019, 7, 17)

datetime.date(2019, 7, 17)

In [17]:
(end - begin).days

6

In [25]:
import twitterscraper.query

In [2]:
def process_tweet_text(text):
    text = re.sub(r'http\S+', '', text)   # Remove URLs
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
    text = text.strip(" ")   # Remove whitespace resulting from above
    text = re.sub(r' +', ' ', text)   # Remove redundant spaces

    # Handle common HTML entities
    text = re.sub(r'&lt;', '<', text)
    text = re.sub(r'&gt;', '>', text)
    text = re.sub(r'&amp;', '&', text)
    return text

In [5]:
import pandas as pd
table = pd.read_csv(r'C:\Users\victo\Desktop\Github Repos\project3\test.csv')

In [7]:
import re
list =[]
for row in table.itertuples():
    process_tweet = process_tweet_text(row.text)
    list.append(process_tweet)

In [35]:
import re

In [8]:
list

['Someone bring me pickles',
 'RT : why the fuck did tommy pickles have a flathead screwdriver. nigga was a whole mechanic',
 'RT : if you ever refer to pickles as “gherkins” around me i will roundhouse kick you in the fucking jaw',
 'RT : Dear pal #AngelicaMaria proves once again why she, "Mexico\'s Sweetheart\' is a living legend. Her comic talents shine in…',
 'Story continues 👨\u200d🍳\r\n\r\n1. Traditional Polish Sourdough Baked in Fermented Cabbage\r\n2. Walnuts ice cream - clear love t…',
 'RT : why the fuck did tommy pickles have a flathead screwdriver. nigga was a whole mechanic',
 'I’m just a boy looking at a girl praying she doesn’t like pickles',
 'RT : why the fuck did tommy pickles have a flathead screwdriver. nigga was a whole mechanic',
 'I want to try these pickle sandwiches😩 the way I love pickles',
 "RT : Hey there! I'm Pickles and I'm a B&W comic artist in need of work. I'm available for projects right now!\r\n\r\n✨Portfolio/Resum…",
 'RT : why the fuck did tommy pic

In [15]:
def generate_tweet(tweet_dict):
    tweet_list =[]
    for row in tweet_dict.itertuples():
        process_tweet = process_tweet_text(row.text)
        tweet_list.append(process_tweet)
    textgen=textgenrnn()
    textgen.train_on_texts(tweet_list, num_epochs=1)
    generated_tweet = textgen.generate(1, temperature=0.5)
    return(generated_tweet)

In [16]:
generate_tweet(table)

Training on 4,025 character sequences.
Epoch 1/1
31/31 [==============================] - ETA: 2:18 - loss: 2.001 - ETA: 1:10 - loss: 2.157 - ETA: 48s - loss: 1.983 - ETA: 36s - loss: 1.92 - ETA: 29s - loss: 1.87 - ETA: 25s - loss: 1.80 - ETA: 21s - loss: 1.75 - ETA: 18s - loss: 1.74 - ETA: 16s - loss: 1.73 - ETA: 14s - loss: 1.68 - ETA: 13s - loss: 1.68 - ETA: 12s - loss: 1.69 - ETA: 11s - loss: 1.68 - ETA: 10s - loss: 1.65 - ETA: 9s - loss: 1.6277 - ETA: 8s - loss: 1.646 - ETA: 7s - loss: 1.632 - ETA: 6s - loss: 1.628 - ETA: 6s - loss: 1.630 - ETA: 5s - loss: 1.616 - ETA: 4s - loss: 1.608 - ETA: 4s - loss: 1.597 - ETA: 3s - loss: 1.588 - ETA: 3s - loss: 1.578 - ETA: 2s - loss: 1.586 - ETA: 2s - loss: 1.590 - ETA: 1s - loss: 1.573 - ETA: 1s - loss: 1.565 - ETA: 0s - loss: 1.553 - ETA: 0s - loss: 1.544 - 13s 420ms/step - loss: 1.5326
####################
Temperature: 0.2
####################
RT : why the fuck did tommy pickles have a whole mechanic

RT : why the fuck did tommy pickles 

In [18]:
textgen.generate(1, temperature=0.1)

RT : why the fuck did tommy pickles  #Pickles of the firste pickles of the flathead screwdriver. nigga was a whole mechanic

